In [1]:
import os
from fnmatch import fnmatch
import numpy as np
from PIL import Image
from filtering.filters import Median
from feature_extraction.lbp import Lbp
from sklearn import svm
import sklearn
import time
import pickle
from sklearn import preprocessing

In [ ]:
#Read features of training samples Gabor
print("loading features")
train_samples_feats_matrix_gabor = np.load("train_samples_feats_matrix_Gabor.npy")

In [ ]:
#Read features of training samples Gabor
print("loading features")
train_samples_feats_matrix_glcm = np.load("train_samples_feats_matrix_GLCM.npy")

In [ ]:
#Load features of validation samples Gabor
valid_samples_feats_matrix_gabor = np.load("valid_samples_feats_matrix_Gabor.npy")

In [ ]:
#Load features of validation samples
valid_samples_feats_matrix_glcm = np.load("valid_samples_feats_matrix_GLCM.npy")

In [ ]:
#concatenating features 
#train_samples_feats_matrix_gabor_glcm = np.concatenate((train_samples_feats_matrix_glcm,train_samples_feats_matrix_gabor), axis=1)
valid_samples_feats_matrix_gabor_glcm = np.concatenate((valid_samples_feats_matrix_glcm,valid_samples_feats_matrix_gabor), axis=1)

In [2]:
test_samples_feats_matrix_GLCM = np.load("test_samples_feats_matrix_GLCM.npy")
test_samples_feats_matrix_Gabor = np.load("test_samples_feats_matrix_Gabor.npy")
test_samples_feats_matrix_glcm_gabor = np.concatenate((test_samples_feats_matrix_GLCM,test_samples_feats_matrix_Gabor), axis=1)

In [3]:
svm_model = pickle.load( open( "SVM_['GLCM', 'Gabor']_[0.2, 0.1]", "rb" ))

In [ ]:
#Train svn model from training data
t0 = time.time()
#nu : upper bound of training errors (normal data but lie outside of normalty boundry)
svm_model = svm.OneClassSVM(kernel='rbf', gamma=0.1, nu=0.1)
svm_model.fit(train_samples_feats_matrix_gabor_LBP)
print("traing time: ", time.time() - t0)

In [4]:
y_predict = svm_model.predict(test_samples_feats_matrix_glcm_gabor)
y_predict_score = svm_model.decision_function(test_samples_feats_matrix_glcm_gabor)

In [ ]:
print len(y_predict)

In [9]:
#Ground truth
y1 = np.repeat(+1, 6000) #normal
y2 = np.repeat(-1, 4000) #abnormal
y = np.concatenate((y1,y2))

In [10]:
f1_binary = sklearn.metrics.f1_score(y, y_predict, pos_label =-1, average = 'binary')
f1_macro = sklearn.metrics.f1_score(y, y_predict, average = 'macro')
auc = sklearn.metrics.roc_auc_score(y, y_predict_score)
Math_Cof = sklearn.metrics.matthews_corrcoef(y, y_predict) 
tn, fp, fn, tp  = sklearn.metrics.confusion_matrix(y, y_predict).ravel()
print ("f1_binary: ", f1_binary, "f1_macro: ", f1_macro, "MAth_cof: ", Math_Cof , " auc: ", auc)
print("confusion matrix", tn, fp, fn, tp)

('f1_binary: ', 0.30627922511690048, 'f1_macro: ', 0.50490952406287504, 'MAth_cof: ', 0.062296408152407348, ' auc: ', 0.58764608333333324)
('confusion matrix', 917, 3083, 1071, 4929)


In [ ]:
print  len(y_predict_score)

In [ ]:
import matplotlib.pyplot as pl
import matplotlib
from skimage import io

fpr,tpr,thresh = sklearn.metrics.roc_curve(y,y_predict_score)

f,ax = pl.subplots(1,1)
ax.plot(fpr,tpr,label="SVM")
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC curves")
ax.legend(loc="lower right")
# these are matplotlib.patch.Patch properties
props = dict(boxstyle='square', facecolor='white', alpha=0.5)

textstr = '$F1-binary=%.2f$\n$F1-macro=%.2f$\n$math-corcoeffient=%.2f$\n$AUC=%.2f$'%(0.65, 0.72, 0.45, 0.72)
# place a text box in upper left in axes coords
ax.text(0.95, 0.2, textstr, transform=ax.transAxes, fontsize=13,
    horizontalalignment='right', verticalalignment='bottom', bbox=props)

io.show()

In [ ]:
f.savefig('SVM_GLCM_Gabor.png')